In [1]:
import requests
import pandas as pd
import plotly.express as px
import json
import sqlalchemy

In [2]:
pd.options.display.max_rows = None

In [3]:
api_key = "9d3018a9c90b575a02fb26df9d7aa565c4e118bb"

In [4]:
cities = ['New York city, New York', 'Los Angeles city, California', 'San Francisco city, California', 'Miami city, Florida', 'Seattle city, Washington', 'Atlanta city, Georgia', 'Denver city, Colorado', 'Chicago city, Illinois', 'Dallas city, Texas']

In [5]:
def json_to_dataframe(response):
    """
    Convert response to dataframe
    """
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [6]:
url = "https://api.census.gov/data/2019/pep/population?get=NAME,POP&for=place:*&key={0}".format(api_key)


In [7]:
response = requests.get(url)

In [8]:
data = response.json()
all_cities = pd.DataFrame(data[1:], columns=data[0])
all_cities["POP"] = all_cities["POP"].astype(float)
all_cities = all_cities.sort_values("NAME")
all_cities.head()

JSONDecodeError: [Errno Expecting value] 
<html>
    <head>
        <title>Invalid Key</title>
    </head>
    <body>
        <p>
            A valid <em>key</em> must be included with each data API request.
            You included a key with this request, however, it is not valid.
            Please check your key and try again.
        </p>
        <p>
            If you do not have a key you may sign up for one <a href="key_signup.html">here</a>.
        </p>
    </body>
</html>
: 1

In [ ]:
filtered_cities = all_cities.loc[all_cities['NAME'].isin(cities)].reset_index()
cities_population = filtered_cities[['NAME', 'POP']]


In [ ]:
cities_population = cities_population.rename(columns = {'NAME':'location', 'POP':'population'}).reset_index(drop=True)
cities_population.head(20)

In [ ]:
cities_population['location'] = cities_population['location'].replace({'New York city, New York':'New York City', 'Los Angeles city, California':'Los Angeles', 'San Francisco city, California':'San Francisco', 'Miami city, Florida':'Miami',
       'Seattle city, Washington':'Seattle', 'Atlanta city, Georgia':'Atlanta', 'Denver city, Colorado':'Denver', 'Chicago city, Illinois':'Chicago', 'Dallas city, Texas':'Dallas'})
cities_population
                                                          

In [ ]:
cities_list = [
    ['New York City', 40.7128, -74.0060],
    ['Los Angeles' , 34.0522, -118.2437],
    ['San Francisco', 37.7749, -122.4194],
    ['Miami', 25.7617, -80.1918],
    ['Seattle', 47.6062, -122.3321],
    ['Atlanta', 33.7490, -84.3880],
    ['Denver', 39.7392, -104.9903],
    ['Chicago', 41.8781, -87.6298],
    ['Dallas', 32.7767, -96.7970]
]

In [ ]:
city_coordinates = {'location':[], 'latitude':[], 'longitude':[]}

In [ ]:
for city in cities_list:
    city_name = city[0]
    latitude = city[1]
    longitude = city[2]
    city_coordinates['location'].append(city[0])
    city_coordinates['latitude'].append(city[1])
    city_coordinates['longitude'].append(city[2])

In [ ]:
cities_cordinates = pd.DataFrame(city_coordinates)

In [ ]:
database_connection_string = 'sqlite:///'

In [ ]:
engine = sqlalchemy.create_engine(database_connection_string)
engine

In [ ]:
cities_population.to_sql('cities_population', engine, index=False, if_exists='replace')

In [ ]:
restaurant_data = pd.read_csv("RestaurantData.csv")

In [ ]:
restaurant_data.to_sql('restaurant_data', engine, index=False, if_exists='replace')

In [ ]:
cities_cordinates.to_sql('cities_cordinates', engine, index=False, if_exists='replace')

In [ ]:
engine.table_names()


In [ ]:
query = """
SELECT rd.location, rd.cuisine, rd.price, rd.count, cp.population, cc.latitude, cc.longitude
FROM restaurant_data AS rd
LEFT JOIN cities_population AS cp
ON rd.location = cp.location
LEFT JOIN cities_cordinates AS cc
ON rd.location = cc.location;
"""

In [ ]:
joined_df = pd.read_sql_query(query, con=engine)

In [ ]:
joined_df.head(40)


In [ ]:
new_join_df = joined_df[['location', 'cuisine', 'price', 'count']]


In [ ]:
pivot_df = joined_df.pivot_table(index=['location', 'cuisine', 'population', 'latitude', 'longitude'], columns='price', values='count')
pivot_df.columns = ['price1', 'price2', 'price3', 'price4']

In [ ]:
pivot_df.reset_index(inplace=True)

In [ ]:
pivot_df

In [ ]:
pivot_df.to_csv('restaurants_pivot.csv')

In [ ]:
pivot_df['total_rest'] = pivot_df['price1'] + pivot_df['price2'] + pivot_df['price3'] + pivot_df['price4']